In [3]:
#neural network model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


In [4]:
df = pd.read_csv('../generated_cpu_data_with_anomaly.csv')
df['date'] = pd.to_datetime(df['date'])

In [5]:
df

,cpu_util,date,anomaly
0,89.447320,2023-06-25 00:03:05.640,1
1,57.320538,2023-06-25 00:08:05.640,0
2,87.058688,2023-06-25 00:13:05.640,1
3,87.409467,2023-06-25 00:18:05.640,1
4,78.111494,2023-06-25 00:23:05.640,1
...,...,...,...
8635,2.692620,2023-07-24 23:38:06.441,0
8636,30.604348,2023-07-24 23:43:06.441,1
8637,14.278655,2023-07-24 23:48:06.441,0
8638,35.247145,2023-07-24 23:53:06.441,1


In [6]:
num_lags = 2

#add lag
for i in range(1, num_lags+1):
    df['lag_{}'.format(i)] = df['cpu_util'].shift(i)
    df['lag_{}_anomaly'.format(i)] = df['anomaly'].shift(i)


In [7]:
df = df.dropna()
df

,cpu_util,date,anomaly,lag_1,lag_1_anomaly,lag_2,lag_2_anomaly
2,87.058688,2023-06-25 00:13:05.640,1,57.320538,0.0,89.447320,1.0
3,87.409467,2023-06-25 00:18:05.640,1,87.058688,1.0,57.320538,0.0
4,78.111494,2023-06-25 00:23:05.640,1,87.409467,1.0,87.058688,1.0
5,65.111306,2023-06-25 00:28:05.640,0,78.111494,1.0,87.409467,1.0
6,94.526610,2023-06-25 00:33:05.640,1,65.111306,0.0,78.111494,1.0
...,...,...,...,...,...,...,...
8635,2.692620,2023-07-24 23:38:06.441,0,16.306464,0.0,30.297529,0.0
8636,30.604348,2023-07-24 23:43:06.441,1,2.692620,0.0,16.306464,0.0
8637,14.278655,2023-07-24 23:48:06.441,0,30.604348,1.0,2.692620,0.0
8638,35.247145,2023-07-24 23:53:06.441,1,14.278655,0.0,30.604348,1.0


In [8]:
#split train and test
X = df[['cpu_util', 'lag_1', 'lag_1_anomaly', 'lag_2', 'lag_2_anomaly']]
y = df['anomaly']

In [9]:
X['cpu_util'] = (X['cpu_util']/100).astype(float)
X['lag_1'] = (X['lag_1']/100).astype(float)
X['lag_2'] = (X['lag_2']/100).astype(float)

C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3277832909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cpu_util'] = (X['cpu_util']/100).astype(float)
C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3277832909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'] = (X['lag_1']/100).astype(float)
C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3277832909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [10]:
X_train = X[:int(0.7*len(X))]
X_test = X[int(0.7*len(X)):]
y_train = y[:int(0.7*len(y))]
y_test = y[int(0.7*len(y)):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3789776307.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y[:int(0.7*len(y))]
C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3789776307.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y[int(0.7*len(y)):]


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(5, 1), return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
189/189 [==============================] - 2s 10ms/step - loss: 0.1306 - accuracy: 0.9484 - val_loss: 0.1048 - val_accuracy: 0.9564
Epoch 2/20
189/189 [==============================] - 2s 10ms/step - loss: 0.1271 - accuracy: 0.9479 - val_loss: 0.1032 - val_accuracy: 0.9564
Epoch 3/20
189/189 [==============================] - 2s 9ms/step - loss: 0.1221 - accuracy: 0.9505 - val_loss: 0.1081 - val_accuracy: 0.9537
Epoch 4/20
189/189 [==============================] - 2s 10ms/step - loss: 0.1219 - accuracy: 0.9510 - val_loss: 0.1057 - val_accuracy: 0.9541
Epoch 5/20
189/189 [==============================] - 2s 9ms/step - loss: 0.1170 - accuracy: 0.9543 - val_loss: 0.1436 - val_accuracy: 0.9533
Epoch 6/20
189/189 [==============================] - 2s 9ms/step - loss: 0.1200 - accuracy: 0.9532 - val_loss: 0.1201 - val_accuracy: 0.9525
Epoch 7/20
189/189 [==============================] - 2s 9ms/step - loss: 0.1179 - accuracy: 0.9550 - val_loss: 0.1059 - val_accuracy: 0.9541
Epo

In [ ]:
# model = tf.keras.models.load_model('../models/nn_with_2gap_2anomalies.h5')

In [19]:
predict = model.predict([
  [0.123, 0.213, 0, 0.9324, 0], 
  [0.502, 0.123, 0, 0.213, 0], 
  [0.8432, 0.502, 0, 0.123, 0], 
  [0.943, 0.8432, 1, 0.502, 0], 
  [0.70, 0.943, 1, 0.8432, 1],
  [1, 0.70, 0, 0.943, 1],
])

1/1 [==============================] - 0s 91ms/step


In [20]:
predict

array([[8.1404378e-06],
       [4.6682647e-01],
       [8.0815858e-01],
       [9.3443632e-01],
       [2.5216900e-02],
       [8.7797117e-01]], dtype=float32)

In [21]:
8.1404378e-06

8.1404378e-06

In [ ]:
array([[4.6048585e-06],
       [3.9161438e-01],
       [7.6850414e-01],
       [5.4467201e-01],
       [2.9657671e-01],
       [5.4232013e-01]], dtype=float32)